In [72]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
    
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sungwoongkang/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sungwoongkang/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [76]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
  separator="\n",
  chunk_size=600,
  chunk_overlap=100,
)

loader = UnstructuredFileLoader('./files/chapter_one.pdf')

loader.load_and_split(text_splitter=splitter)

[Document(page_content='chapter_one.md\n2024-04-06\nPart 1, Chapter 1\nPart One\n1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into\nhis breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions,\nthough not quickly enough to prevent a swirl of gritty dust from entering along with him.\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for\nindoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide:', metadata={'source': './files/chapter_one.pdf'}),
 Document(page_content='the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features.\nWinston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working,\nand at present the electric current was cut off during daylight hours. It was part of the economy drive in